<a href="https://colab.research.google.com/github/jihyunjeongme/deeplearning-tensflow-3min/blob/master/%5B3%EB%B6%84_%EB%94%A5%EB%9F%AC%EB%8B%9D%5D_5%EC%9E%A5_%ED%85%90%EC%84%9C%EB%B3%B4%EB%93%9C%EC%99%80_%EB%AA%A8%EB%8D%B8_%EC%9E%AC%EC%82%AC%EC%9A%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5.1 학습 모델 저장하고 재사용하기

In [57]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [48]:
cd /content/gdrive/My Drive/Colab Notebooks/deeplearning-tensorflow-3mins

/content/gdrive/My Drive/Colab Notebooks/deeplearning-tensorflow-3mins


In [0]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])


In [0]:
# loadtxt 함수를 이용하여 간단하게 데이터를 읽어들인 후 
# 1열과 2열은 x_data로, 3열부터 마지막 열까지는 y_data로 변환하였음.

In [0]:
# 신경망 모델을 구성할 차례
# 학습 횟수를 카운트 하는 변수(학습에 직접 사용되지는 않음)
# 변수 정의 시 trainable=False라는 옵션을 줌

global_step = tf.Variable(0, trainable=False, name='global_step')


In [0]:
# 앞 장에서 보다 계층을 하나 더 늘리고, 편향은 없이 가중치만 사용한 모델로 만들어 봄
# 계층은 하나 늘었지만, 모델이 더 간략해졌으니 신경망의 구성이 조금 더 명확하게 드러남

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)\

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

cost = tf.reduce_mean(
tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,
                                          logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(cost, global_step=global_step)

In [0]:
# 코드를 보면 두 번째 가중치인 W2의 형태가 [10, 20]인것을 볼 수있음.
# 이유는 앞단 계층의 출력 크기가 10이고, 뒷단 계층의 입력 크기가 20이기 때문
# global_step 매개변수에 앞서 정의한 global_step변수를 넘겨준 것을 확인
# 이렇게 하면 최적화 함수가 학습용 변수들을 최적화 할때마다
# global_step 변수의 값을 1씩 증가시키게 됨

In [0]:
# 모델 구성 끝
# 세션을 열고 최적화를 실행
# 모델을 불러들이고 저장하는 코드

########
# 신경망 모델 학습
#######

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

In [0]:
# tf.global_variables는 앞서 정의한 변수들을 가져오는 함수
# 이 함수를 써서 앞서 정의한 변수들을 모두 가져와서, 이후 변수들을 파일에 저장하거나
# 이전에 학습한 결과를 불러와 담는 변수들로 사용

In [0]:
# ./model 디렉터리에 기존에 학습해둔 모델이 있는지를 확인해서 
# 모델이 있다면 saver.restore 함수를 사용해 학습된 값들을 불러오고,
# 아니면 변수를 새로 초기화
# 학습된 모델을 저장한 파일을 체크포인트 파일 이라고 함

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
  saver.restore(sess, ckpt.model_checkpoint_path)
else:
  sess.run(tf.global_variables_initializer())
  

In [24]:
# 최적화 수행
# 이전과는 달리 step 값이 아니라 global_step 값을 이용해 
# 학습을 몇 번째 진행하고 있는지를 출력
# global_step은 텐서 타입의 변수이므로 값을 가져 올 때 sess.run(global_step)을 이용

for step in range(2):
  sess.run(train_op, feed_dict={X: x_data, Y:y_data})
  
  print('Step: %d, ' % sess.run(global_step),
        'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
  

Step: 9,  Cost: 0.781
Step: 10,  Cost: 0.728


In [29]:
# 마지막으로 최적화가 끝난 뒤 학습된 변수들을 지정한 체크포인트 파일에 저장
# 두 번쨰 매개변수는 체크포인트 파일의 위치와 이름
# global_step의 값은 저장할 파일의 이름을 추가로 붙게 되며
# 텐서 변수 또는 숫자값을 넣어줄 수 있습니다.
# 이를 이용해 여러 상태의 체크포인트를 만들 수 있고, 가장 효과적인 체크포인트를 선별,사용


saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-10'

In [30]:
# 예측 결과와 정확도 확인

prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data,
                                                       Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]
정확도: 100.00


In [0]:
# 총 10번을 실했해을 때 정확도가 100.00으로 나옴
# gobal_step으로 저장한 값을 불러와서 증가시키면 step이 늘어남
# 텐서플로를 이용하면 이렇게 쉽게 학습시킨 모델을 저장하고 불러와서
# 재사용 할수 있음.
# 이 방식을 이용해 모델 구성, 학습, 예측 부분을 각각 분리하여 학습을 따로 한 뒤
# 예측만 단독으로 실행하는 프로그램을 작성할 수 있음

# 5.2 텐서보드 사용하기

In [0]:
# 텐서보드는 학습 과정을 추정하는 데 용이
# 학습하는 중간중간 손실값이나 정확도 또는 결로 나온 이미지나 사운드 파일들을
# 다양한 방식으로 시각화해 보여줌

In [0]:
# 먼저 데이터를 읽어 들이는 코드와 플레이스홀더 값들을 똑같이 넣어줌

import tensorflow as tf
import numpy as np

data = np.loadtxt('./data.csv', delimiter=',',
                 unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])

global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [0]:
# 신경망의 각 계층에 다음 코드를 덧붙여 줍니다.
# with tf.name_scope로 묶은 블록은 텐서보드에서 한 계층 내부를 표현
# name='W1' 이름 붙이면 텐서보드에서 해당 이름의 변수가 어디서 사용되는지 쉽게 확인
# 이름은 변수뿐만 아니라 플레이스 홀더, 각각의 연산, 활성화 함수 등 모든 텐서에 붙일 수 있음


with tf.name_scope('layer1'):
  W1 = tf.Variable(tf.random_uniform([2,10], -1., 1.), name='W1')
  L1 = tf.nn.relu(tf.matmul(X,W1))
  

In [0]:
# 이렇게 다른 계층들도 전부 tf.name_scope로 묶어주고 이름도 붙여줌

with tf.name_scope('layer2'):
  W2 = tf.Variable(tf.random_uniform([10,20], -1., 1.), name='W2')
  L2 = tf.nn.relu(tf.matmul(L1,W2))
  
with tf.name_scope('output'):
  W3 = tf.Variable(tf.random_uniform([20,3], -1., 1.), name='W3')
  model = tf.matmul(L2,W3)
  
with tf.name_scope('optimizer'):
  cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y,
                                               logits=model))
  
  optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
  train_op = optimizer.minimize(cost, global_step=global_step)
    

In [70]:
# 손실값을 추적하기 위해 수집할 값을 지정하는 코드 작성
# tf.summary 모듈의 scalar 함수는 값이 하나인 텐서를 수집할 떄 사용

tf.summary.scalar('cost',cost)

<tf.Tensor 'cost_3:0' shape=() dtype=string>

In [0]:
# 모델을 불러들이거나 초기화

sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
  saver.restore(sess, ckpt.model_checkpoint_path)
else:
  sess.run(tf.global_variables_initializer())
  

In [0]:
# tf.summary.merge_all 함수로 앞서 지정한 텐서들을 수집한 다음
# tf.summary.FileWriter 함수를 이용해 그래프와 텐서들의 값을 저장할 디텍터리 설정

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

In [73]:
# 최적화를 실행하는 코드를 앞서와 같이 작성

for step in range(100):
  sess.run(train_op, feed_dict={X: x_data, Y:y_data})
  
  print('Step: %d, ' % sess.run(global_step),
        'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))
  

Step: 1,  Cost: 1.667
Step: 2,  Cost: 1.497
Step: 3,  Cost: 1.344
Step: 4,  Cost: 1.210
Step: 5,  Cost: 1.099
Step: 6,  Cost: 1.010
Step: 7,  Cost: 0.946
Step: 8,  Cost: 0.900
Step: 9,  Cost: 0.864
Step: 10,  Cost: 0.835
Step: 11,  Cost: 0.812
Step: 12,  Cost: 0.790
Step: 13,  Cost: 0.768
Step: 14,  Cost: 0.747
Step: 15,  Cost: 0.728
Step: 16,  Cost: 0.710
Step: 17,  Cost: 0.693
Step: 18,  Cost: 0.677
Step: 19,  Cost: 0.663
Step: 20,  Cost: 0.649
Step: 21,  Cost: 0.636
Step: 22,  Cost: 0.624
Step: 23,  Cost: 0.614
Step: 24,  Cost: 0.605
Step: 25,  Cost: 0.598
Step: 26,  Cost: 0.592
Step: 27,  Cost: 0.587
Step: 28,  Cost: 0.582
Step: 29,  Cost: 0.578
Step: 30,  Cost: 0.575
Step: 31,  Cost: 0.572
Step: 32,  Cost: 0.570
Step: 33,  Cost: 0.567
Step: 34,  Cost: 0.566
Step: 35,  Cost: 0.564
Step: 36,  Cost: 0.563
Step: 37,  Cost: 0.561
Step: 38,  Cost: 0.560
Step: 39,  Cost: 0.559
Step: 40,  Cost: 0.558
Step: 41,  Cost: 0.558
Step: 42,  Cost: 0.557
Step: 43,  Cost: 0.556
Step: 44,  Cost: 0.5

In [74]:
# sess.run을 이용해 앞서 merged로 모아둔 텐서의 값을 계산하여 수집한 뒤
# writer.add_summary 함수를 이용해 해당 값들을 앞서 지정한 디텍터리에 저장
# 적절한 시점에(여기서는 매 단계에서) 값들을 수집하고 저장하면 되며
# 나중에 확인할 수 있도록 global_step 값을 이용해 수집한 시점을 기록해둠

summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
writer.add_summary(summary, global_step=sess.run(global_step))

InvalidArgumentError: ignored